### Data Cleaning / Merging

In [1]:
# some imports
import pandas as pd
import numpy as np
import csv
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# load data (converted from json to csv using yelp git repo converter) (takes a few min)
# data structure: https://www.yelp.co.uk/dataset_challenge/dataset
business = pd.read_csv('yelp_academic_dataset_business.csv')
#review = pd.read_csv('yelp_academic_dataset_review.csv')
#user = pd.read_csv('yelp_academic_dataset_user.csv')

c:\Users\Jeremy\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (1,4,7,14,18,21,27,30,43,49,52,62,64,66,70,84,89,92,100) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
# merge datasets
#merged = pd.merge(review,business,on='business_id')

In [4]:
subset = business[['name', 'state', 'city', 'stars', 'categories']]

In [6]:
# Figure out which attributes are boolean and which are values
cols = [x for x in business.columns.tolist() if 'attributes' in x]

In [7]:
# Handle these columns specially

# First set have a "false" value
special_cols = {
    'attributes.Alcohol': 'none',
    'attributes.BYOB/Corkage': 'no',
    'attributes.Wi-Fi': 'no',
    'attributes.Smoking': 'no'
}

# Second set don't
special_cols_2 = [
    'attributes.Ages Allowed',
    'attributes.Price Range',
    'attributes.Noise Level',
    'attributes.Attire'
]

In [8]:
merged = business

In [9]:
# Create a single attributes column
subset['attributes'] = ""

# For each column
for c in cols:
    # Don't add for null rows
    b = ~pd.isnull(merged[c])
    
    if c in special_cols:
        b = b&(merged[c] != special_cols[c])
        subset['attributes'][b] += ";" + c[11:] + '=' + merged[c][b].astype(str)
    elif c in special_cols_2:
        subset['attributes'][b] += ";" + c[11:] + '=' + merged[c][b].astype(str)
    else:
        b = b & (merged[c].astype(bool))
        subset['attributes'][b] += ";" + c[11:]
        
# Strip leading semicolon
subset['attributes'][subset['attributes'] != ""] = subset['attributes'][subset['attributes'] != ""].str[1:]

c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from IPython.kernel.zmq import kernelapp as app
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Jeremy\Anaconda\lib\site-packages\pandas\core\generic.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inpla

In [10]:
# pivot out categories column 
import ast

L = [] # empty list to store categories

for l in subset.categories.unique():
    L.extend(ast.literal_eval(l)) # literal turns string list into list list

print(len(set(L)), 'categories')
cat = list(set(L)) # store distinct list

(783, 'categories')


In [22]:
subset['categories_new'] = ""
for c in cat:
    b = subset['categories'].str.contains(c)
    subset['categories_new'][b] += ";" + c
    
# Strip leading semicolon
subset['categories_new'][subset['categories_new'] != ""] = subset['categories_new'][subset['categories_new'] != ""].str[1:]

c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\Users\Jeremy\Anaconda\lib\site-packages\pandas\core\strings.py:207: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  " groups, use str.extract.", UserWarning)
c:\Users\Jeremy\Anaconda\lib\site-packages\IPython\kernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on

In [23]:
subset[subset['categories'].str.contains('Pizza') & (subset.city == 'Charlotte')].sample(1)

,name,state,city,stars,categories,attributes,categories_new
33441,Hungry Howie's Pizza,NC,Charlotte,2.5,"['Pizza', 'Sandwiches', 'Restaurants']",Outdoor Seating;Parking.lot;Accepts Credit Car...,Restaurants;Pizza;Sandwiches


In [29]:
scores = [1,2,5]
temps = [1,5,6,7,8,9]

for score in scores:
    for temp in temps:
        reviews = []
        with open('output/pizza_%s_temp_%s' % (score, temp)) as f:
            for l in f.readlines()[1:-1]:
                if len(l) < 300 and len(l) > 2:
                    reviews.append(l.replace('\n', ' ').strip())

        bus = subset[subset['categories'].str.contains('Pizza') & (subset.city == 'Charlotte')].sample(len(reviews))
        bus['review_stars'] = score
        bus['text'] = reviews

        bus[
            ['name', 'state', 'city', 'stars', 'attributes', 'categories_new', 'review_stars', 'text']
        ].rename(columns={
            'stars': 'business_stars', 'categories_new': 'categories'
        }).to_csv('fake/pizza_%s_temp_%s.csv' % (score, temp), sep = '\t', index=False, quoting=csv.QUOTE_NONE, header=False)